# Build dataset to feed ShowStats Dash application 

## Imports

In [45]:
import pandas as pd
import datetime
pd.options.display.max_columns = 200
import numpy as np
import requests
import json
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.express as px
import warnings
import dash_bootstrap_components as dbc
warnings.simplefilter(action='ignore', category=FutureWarning)

## API https://relistenapi.alecgorge.com/api/

In [57]:
new_df = pd.read_parquet(path="/Users/brandonmurphy/projects/show_stats/ShowStats/data/showstats.parquet")

new_df.sample(15)

,artist_id,venue_id,tour_id,uuid,id,date,avg_rating,avg_duration,display_date,venue.shows_at_venue,venue.latitude,venue.longitude,venue.name,venue.location,venue.slug,venue.past_names,venue.id,year.year,artist,source_uuid,show_uuid,track_position,duration,title,slug,mp3_url
11683,9,59031.0,1364.0,bc0a4100-4ace-868a-aaa7-e384d93659de,24033979,1977-05-26T00:00:00Z,9.085714,10915.0,1977-05-26,6.0,NaN,NaN,Baltimore Civic Center,"Baltimore, MD, USA",baltimore-civic-center,NaN,59031.0,1977,Grateful Dead,c6ca08fe-7497-d6a9-34a8-185ff5d7e28a,bc0a4100-4ace-868a-aaa7-e384d93659de,6.0,239.0,PASSENGER,passenger,https://archive.org/download/gd77-05-26.sbd.sa...
11287,9,58921.0,1364.0,adfec66e-9c09-5123-4a24-e5d8d60b4bb9,24033959,1977-04-27T00:00:00Z,8.432432,10514.0,1977-04-27,23.0,NaN,NaN,Capitol Theater,"Late Show Port Chester, NY, USA",capitol-theater,NaN,58921.0,1977,Grateful Dead,c2f5340f-235f-383b-f6f9-c5ffe811d258,adfec66e-9c09-5123-4a24-e5d8d60b4bb9,2.0,798.0,MISSISSIPPI HALF STEP,mississippi-half-step,https://archive.org/download/gd77-04-27.sbd.sa...
7621,5,85061.0,2292.0,b295fe16-980f-31cc-3a0b-99f00e61f582,24116887,1994-10-26T00:00:00Z,0.000000,8717.0,1994-10-26,5.0,39.961176,-82.998794,Newport Music Hall,"Columbus, Ohio",newport-music-hall,NaN,85061.0,1994,Widespread Panic,63d494a6-4f4a-9191-8eca-280b3d5da53e,b295fe16-980f-31cc-3a0b-99f00e61f582,11.0,338.0,AIN'T LIFE GRAND,aint-life-grand,https://www.panicstream.com/streams/wsp/1994_1...
81550,4,83379.0,2689.0,9596b267-1f2c-1d9f-620d-463f8a4a01a0,121539530,2022-12-31T00:00:00Z,8.932000,13373.0,2022-12-31,72.0,40.750505,-73.993439,Madison Square Garden,"New York, NY",madison-square-garden,NaN,83379.0,2022,Phish,2ab1bbd2-cc8e-d281-6d80-02881f7a04ba,9596b267-1f2c-1d9f-620d-463f8a4a01a0,3.0,663.0,SET YOUR SOUL FREE,set-your-soul-free,https://phish.in/audio/000/036/107/36107.mp3
33171,9,58702.0,NaN,b74118f9-cbdf-b49b-a8da-4da5a5153045,24035406,1992-02-21T00:00:00Z,9.111111,2815.0,1992-02-21,6.0,NaN,NaN,Club Front Studio,"San Rafael, CA",club-front-studio,NaN,58702.0,1992,Grateful Dead,438230d2-285b-2089-9fc5-80ecaef228a4,b74118f9-cbdf-b49b-a8da-4da5a5153045,11.0,70.0,PHIL TEACHESE TO THE WIND CHORDS TO BAND,phil-teaches-wave-to-the-wind-chords-to-band,https://archive.org/download/gd92-02-21.smr-re...
38452,202,82120.0,NaN,3c283884-88bc-549b-5bd3-4f1d601d3185,24107074,2017-02-16T00:00:00Z,0.000000,6476.0,2017-02-16,1.0,NaN,NaN,"Bunk Bar, Portland, OR",Portland OR,bunk-bar-portland-or,NaN,82120.0,2017,Billy Strings,25ca6a9e-8454-c69a-34f2-290ff8ac9d31,3c283884-88bc-549b-5bd3-4f1d601d3185,3.0,326.0,BS 20170216_03,bs-2017-02-16-03flac,https://archive.org/download/billystrings2017-...
50084,4,83311.0,2203.0,59a19e08-7291-9561-37d6-cfe70c2c4a83,24108269,1989-08-17T00:00:00Z,6.400000,9694.0,1989-08-17,27.0,44.483316,-73.220556,The Front,"Burlington, VT",the-front,NaN,83311.0,1989,Phish,ed7cfc4c-5846-ab64-6577-4d32296955b1,59a19e08-7291-9561-37d6-cfe70c2c4a83,10.0,234.0,WALK AWAY,walk-away,https://phish.in/audio/000/011/416/11416.mp3
43120,202,98639.0,NaN,a8352da6-0bf6-f900-2221-657202f288ec,89156822,2021-12-09T00:00:00Z,0.000000,7946.0,2021-12-09,4.0,NaN,NaN,The Eastern,"Atlanta, Georgia",the-eastern,NaN,98639.0,2021,Billy Strings,19b3a1d9-4b1e-0999-a51c-4673106dd88f,a8352da6-0bf6-f900-2221-657202f288ec,7.0,530.0,THIRST MUTILATOR,thirst-mutilator,https://archive.org/download/billystrings2021-...
72320,4,82782.0,2173.0,7b263099-3cab-c260-8572-2d821b634555,24109418,2010-08-14T00:00:00Z,8.770000,9977.0,2010-08-14,23.0,42.785292,-88.405096,Alpine Valley Music Theatre,"East Troy, WI",alpine-valley-music-theatre,NaN,82782.0,2010,Phish,fe60f8f0-5e47-f7f3-d6cd-1627ce8583fc,7b263099-3cab-c260-8572-2d821b634555,19.0,448.0,MIKE'S SONG,mikes-song,https://phish.in/audio/000/023/263/23263.mp3
19117,9,58442.0,NaN,9e7e828c-4a61-55f1-02fb-b0126499cf3e,24035383,1982-01-01T00:00:00Z,5.333334,2535.0,1982-01-01,1.0,NaN,NaN,Mickey's ranch studio,"Novato, CA",mickeys-ranch-studio,NaN,58442.0,1982,Grateful Dead,e

In [58]:
new_df.groupby(['artist'])['uuid'].nunique()

artist
Billy Strings        383
Goose                146
Grateful Dead       2063
Phish               1711
Widespread Panic    2356
Name: uuid, dtype: int64

In [46]:
slugs = [
    'grateful-dead',
    'phish',
    'billy-strings',
    'billy-and-the-kids',
    'goose',
    'greensky-bluegrass',
    'jrad',
    'spafford',
    'sci',
    'umphreys',
    'wsp', 
]

## Function to Return All Shows for Selected Bands

In [50]:
artist_id_mapping = {5:'Widespread Panic',
                     4:'Phish',
                     259:'Goose',
                     202: "Billy Strings",
                     9: "Grateful Dead"
                     }

In [52]:
def get_show_dates(slug_list: list[str]):
    show_dates = []
    for slug in slug_list:
        print(slug)
        # Find the years that a band has been active (based on relisten.net reference)
        url = f'https://relistenapi.alecgorge.com/api/v2/artists/{slug}/years'
        response = requests.get(url)
        years = pd.json_normalize(response.json())
        active_years = years['year'].unique() 
        
        # Find the dates of shows within each of the years returned from above
        for year in active_years:
            url_years = f'https://relistenapi.alecgorge.com/api/v2/artists/{slug}/years/{year}'
            response = requests.get(url_years)
            years = pd.json_normalize(response.json(), record_path=['shows'])
            show_dates.append(years)
    
    select_cols = ['artist_id',
                   'venue_id', 
                   'tour_id',
                   'uuid',
                   'id',
                   'date',
                   'avg_rating',
                   'avg_duration',
                   'display_date',
                   'venue.shows_at_venue',
                   'venue.latitude',
                   'venue.longitude',
                   'venue.name',
                   'venue.location',
                   'venue.slug',
                   'venue.past_names',
                   'venue.id',
                   'year.year'
                ]
    tour_dates = pd.concat(show_dates).drop_duplicates()
    tour_dates = tour_dates[select_cols]
    tour_dates['artist'] = tour_dates['artist_id'].map(artist_id_mapping)
        
    return tour_dates

In [53]:
def get_uuid_list(show_date_df: pd.DataFrame):
    uuid_list = show_date_df['uuid'].unique().tolist()
    return uuid_list

In [69]:
all_show_dates = get_show_dates(slug_list=['wsp', 'phish', 'goose']) #

wsp
phish
goose


In [ ]:
all_show_dates.artist.unique()

In [ ]:
all_uuids = get_uuid_list(all_show_dates)

## Function to Return All Songs in Selected Shows

In [ ]:
# Need to handle:
# Songs that get sandwiched
# Mispelled song titles
# Stuff like "jam", or "tuning"
# Missing venue information 

def get_show_songs(uuid_list = list[str]):
    setlist_tracks = []
    for show_uuid in uuid_list:
        # Make API call, and flatten dataframe
        url = f'https://relistenapi.alecgorge.com/api/v3/shows/{show_uuid}'
        response = requests.get(url)
        show_json = response.json()
        show_records = pd.json_normalize(data = show_json, record_path=['sources', 'sets', 'tracks'])
        
        # Select Columns, only take 1st recording of a given show
        show_records['show_uuid'] = show_uuid
        select_cols = ['source_uuid', 'show_uuid', 'track_position', 'duration', 'title', 'slug', 'mp3_url']
        main_taper_id = show_records['source_uuid'].unique().tolist()[0]
        show = show_records[show_records['source_uuid'] == main_taper_id][select_cols]
        
        # Drop duplicates, push song titles to uppercase, filter out songs that arent actually songs
        show = show.drop_duplicates(subset = ['title'])
        show['title'] = show['title'].str.upper()
        keywords = ['JAM', 'BANTER', 'TUNING', 'INTRO', 'GREETING',]
        show = show[~show['title'].str.contains('|'.join(keywords))]
        
        # Clean up song titles (remove special characters, remove file formats (.wav, .flac))
        show['title'] = show['title'].str.replace("^\d+\s+", "", regex = True)
        show['title'] = show['title'].str.replace(".WAV|.FLAC|-|>|<|[|]", "", regex = True)
        setlist_tracks.append(show)
    
    setlist_final = pd.concat(setlist_tracks)
        
    return setlist_final    

## Combine Shows to Tracklists

In [ ]:
all_tracks = get_show_songs(all_uuids)

In [ ]:
final_df = all_show_dates.merge(all_tracks, left_on = 'uuid', right_on = 'show_uuid', how = 'left')

In [ ]:
final_df.artist.unique()

In [ ]:
#final_df.to_parquet(path = "/Users/brandonmurphy/projects/show_stats/ShowStats/data/wsp_phish_goose_test_df.parquet")

In [3]:
final_df = pd.read_parquet(path = "/Users/brandonmurphy/projects/show_stats/ShowStats/data/wsp_phish_goose_test_df.parquet")
final_df.head()

,artist_id,venue_id,tour_id,uuid,id,date,avg_rating,avg_duration,display_date,venue.shows_at_venue,venue.latitude,venue.longitude,venue.name,venue.location,venue.slug,venue.past_names,venue.id,year.year,artist,source_uuid,show_uuid,track_position,duration,title,slug,mp3_url
0,5,85521.0,2292.0,292236fc-34dd-21e4-1e4d-eb5fc7dc081e,24116442,1985-06-22T00:00:00Z,0.0,1955.0,1985-06-22,43.0,33.960948,-83.377936,Uptown Lounge,"Athens, Georgia",uptown-lounge,NaN,85521.0,1985,Widespread Panic,9ed2a46e-fb09-d1e6-7d5f-80cbc4fac1c1,292236fc-34dd-21e4-1e4d-eb5fc7dc081e,1.0,310.0,LOVE THE ONE YOU'RE WITH,love-the-one-youre-with,https://www.panicstream.com/streams/wsp/1985_0...
1,5,85521.0,2292.0,292236fc-34dd-21e4-1e4d-eb5fc7dc081e,24116442,1985-06-22T00:00:00Z,0.0,1955.0,1985-06-22,43.0,33.960948,-83.377936,Uptown Lounge,"Athens, Georgia",uptown-lounge,NaN,85521.0,1985,Widespread Panic,9ed2a46e-fb09-d1e6-7d5f-80cbc4fac1c1,292236fc-34dd-21e4-1e4d-eb5fc7dc081e,2.0,103.0,STIR IT UP,stir-it-up,https://www.panicstream.com/streams/wsp/1985_0...
2,5,85521.0,2292.0,292236fc-34dd-21e4-1e4d-eb5fc7dc081e,24116442,1985-06-22T00:00:00Z,0.0,1955.0,1985-06-22,43.0,33.960948,-83.377936,Uptown Lounge,"Athens, Georgia",uptown-lounge,NaN,85521.0,1985,Widespread Panic,9ed2a46e-fb09-d1e6-7d5f-80cbc4fac1c1,292236fc-34dd-21e4-1e4d-eb5fc7dc081e,3.0,230.0,THE OTHER ONE,the-other-one,https://www.panicstream.com/streams/wsp/1985_0...
3,5,85521.0,2292.0,292236fc-34dd-21e4-1e4d-eb5fc7dc081e,24116442,1985-06-22T00:00:00Z,0.0,1955.0,1985-06-22,43.0,33.960948,-83.377936,Uptown Lounge,"Athens, Georgia",uptown-lounge,NaN,85521.0,1985,Widespread Panic,9ed2a46e-fb09-d1e6-7d5f-80cbc4fac1c1,292236fc-34dd-21e4-1e4d-eb5fc7dc081e,4.0,69.0,CRYPTICAL ENVELOPMENT,cryptical-envelopment,https://www.panicstream.com/streams/wsp/1985_0...
4,5,85521.0,2292.0,292236fc-34dd-21e4-1e4d-eb5fc7dc081e,24116442,1985-06-22T00:00:00Z,0.0,1955.0,1985-06-22,43.0,33.960948,-83.377936,Uptown Lounge,"Athens, Georgia",uptown-lounge,NaN,85521.0,1985,Widespread Panic,9ed2a46e-fb09-d1e6-7d5f-80cbc4fac1c1,292236fc-34dd-21e4-1e4d-eb5fc7dc081e,5.0,520.0,FIRE ON THE MOUNTAIN,fire-on-the-mountain,https://www.panicstream.com/streams/wsp/1985_0...


## Plots to Prototype Dash App Div's

In [4]:
shows_ive_seen = [
                  # Widespread Panic
                  '2019-04-27', '2021-07-16', '2021-07-17', '2021-07-18',
                  '2021-08-08', '2022-02-18', '2022-02-19', '2022-02-20',
                  '2022-03-26', '2022-05-06', '2022-05-07',
                  
                  # Phish
                  '2019-06-21', '2019-12-06', '2019-12-07', '2021-07-31',
                  '2021-08-01', '2022-05-31', '2022-06-01', '2022-07-29',
                  
                  # Billy Strings 
                  #'2021-04-03', '2022-06-24', '2022-06-25', # Missing - Austin, Greensboro, 1 Cola Show, 1 Cary, 
                  
                  # Goose
                  '2020-10-17','2021-05-06', '2022-06-15', '2022-10-15',# Missing charlotte and charleston 
                  
                  ]

In [18]:
my_shows = final_df[final_df['display_date'].isin(shows_ive_seen)].copy()

In [19]:
my_shows.drop_duplicates(subset=["display_date"])[['uuid', 'artist']]

,uuid,artist
45975,1ef462cc-7647-878f-78b0-4a7f635b5c52,Widespread Panic
46858,8caf7ca3-21e9-d1e6-4ad3-760a2565e2b3,Widespread Panic
46880,f7f33882-abc8-314f-5cad-33811c74525e,Widespread Panic
46901,ff62abb3-8327-93df-5db5-d02a682b375b,Widespread Panic
46966,715e7c96-0def-9781-9bae-0b1a53510b62,Widespread Panic
47388,d599090e-1d55-8e9a-8557-113bc3fd3353,Widespread Panic
47408,8a890c21-2732-cbbe-a596-8d8958485e27,Widespread Panic
47429,cd37e7c4-1c5a-b17f-30ea-515189dd876e,Widespread Panic
47534,5cee6580-4d79-8cdd-1f0a-30b7fcca66aa,Widespread Panic
47580,fb84061b-5564-3f6a-b6f5-ee0ffeecbb55,Widespread Panic


In [8]:
my_uuids = ['1ef462cc-7647-878f-78b0-4a7f635b5c52',
            '8caf7ca3-21e9-d1e6-4ad3-760a2565e2b3',
            'f7f33882-abc8-314f-5cad-33811c74525e',
            'ff62abb3-8327-93df-5db5-d02a682b375b',
            '715e7c96-0def-9781-9bae-0b1a53510b62',
            'd599090e-1d55-8e9a-8557-113bc3fd3353',
            '8a890c21-2732-cbbe-a596-8d8958485e27',
            'cd37e7c4-1c5a-b17f-30ea-515189dd876e',
            '5cee6580-4d79-8cdd-1f0a-30b7fcca66aa',
            'fb84061b-5564-3f6a-b6f5-ee0ffeecbb55',
            '0be926ad-7264-faa4-b9d4-807d2532f495',
            '11005e2a-46b7-288a-d0fb-5281e6730f51',
            'e588f4ea-be66-3924-419e-3085fff17a4e',
            '72507b40-4b02-fc82-109a-b919566976f4',
            '4a5d66c7-1c9b-6eda-e198-1bd3f4cdcfd6',
            '13367854-2d61-402d-4e35-ac76af4042cd',
            '819e3062-1b81-250e-1c19-6b8f66e87718',
            'b8b481fd-bc0d-f9ac-7202-5e4185a163ec',
            'df8da48f-7c7c-0273-ea9d-9b7bc877c6d1',
            '821f67a4-2a5c-85e5-6fdd-4503b6cf7bbc',
            '41a9fa41-de1c-dc50-1a78-73350ad1766b',
            '0484ba95-27d1-314e-4ab6-9923ca3105a3',
            'b8a97c8d-8850-6f9b-6b6a-792b067c8c10',
            '2acafa23-1821-e5dd-f2b8-a680c73a92db'
    ]

#### What Songs Have I Seen Most?

In [9]:
counts = (my_shows
 .groupby(['artist', 'title'])['slug']
 .count()
 .reset_index()
 .sort_values("slug", ascending = False)
 .rename(columns = {"slug":"times_heard"})
 .head(25)
 .copy()
 )

In [ ]:
bar = px.bar(data_frame = counts,
             x = 'times_heard',
             y = 'title',
             color = 'artist',
             title = "Songs You've Seen Played Most",
             text = 'times_heard',
             labels = {'title':'', 'times_heard':'Number Times Seen', 'artist':'Artist'}
             )
bar.update_layout(yaxis={'categoryorder':'total ascending'})
bar.update_traces(textposition='outside')
pyo.plot(bar)

#### How Many Unique Songs Have I Seen?

In [ ]:
unique_songs = (my_shows
                .groupby(['artist'])['title']
                .nunique()
                .reset_index()
                .copy()
                )
unique_songs

In [ ]:
plot2 = px.bar(data_frame=unique_songs,
               x = 'artist',
               y = 'title',
               color = 'artist',
               title = "Number of Unique Songs Seen",
               text = 'title',
               labels = {'artist': 'Artist',
                        'title': 'Unique Songs Seen',
                        }
               )
plot2.update_traces(textposition='outside')

pyo.plot(plot2)

#### Map Locations of Venues I've Seen 

In [11]:
def convert_seconds_to_hms(df, column):
    df['duration_hms'] = df[column].astype('datetime64[s]').dt.strftime("%H:%M:%S")
    return df

my_shows = convert_seconds_to_hms(my_shows, 'avg_duration')
    

In [13]:
fig = px.scatter_mapbox(my_shows,
                        lat = "venue.latitude",
                        lon="venue.longitude",
                        hover_name="artist",
                        hover_data=["venue.name", "venue.location", 'display_date', 'duration_hms'],
                        color='artist',
                        zoom=3,
                        height=550,
                        opacity = .6,
                        title = "Locations of Concerts I've Seen",
                        labels = {"venue.name":'Venue',
                                  'venue.location':'Location',
                                  'venue.latitude':'Latitude',
                                  'venue.longitude':'Longitude',
                                  'artist':'Artist',
                                  'display_date':'Date',
                                  'duration_hms':"Show Time"
                                  },
                        size_max = 100
                        )
fig.update_traces(marker={'size': 12})
fig.update_layout(mapbox_style="open-street-map") # carto-darkmatter
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
pyo.plot(fig)

'temp-plot.html'

### Show Count by Year / Band

In [ ]:
shows_by_year = (my_shows
                 .groupby(['year.year', 'artist'])['uuid']
                 .nunique()
                 .reset_index()
                 .sort_values(by = ['year.year', 'artist'])
                 )
year_order = shows_by_year['year.year'].sort_values(ascending=True).unique()
category_order = {'year.year':year_order}

In [ ]:
px.line(data_frame = shows_by_year,
        x = 'year.year',
        y = 'uuid',
        color = 'artist',
        markers = True,
        category_orders = category_order,
        title = 'Show Counts by Artist / Year',
        labels = {'year.year':'Year', 'uuid':'Show Count', 'artist':'Artist'}
    )

### Longest Jams I've Seen?

In [10]:
longest_jams = (
    my_shows
    .sort_values("duration", ascending = False)
    .assign(duration_hms = my_shows['duration'].astype('datetime64[s]').dt.strftime("%M:%S"))
    .assign(minutes = round(my_shows['duration'] / 60, 2))
    .assign(city = my_shows['venue.location'].str.split(pat = ",", n = 1).str[0])
    .head(20)
)
longest_jams = longest_jams[['artist', 'venue.location', 'title', 'duration_hms', 'display_date', 'track_position']]
#longest_jams['title_year_city'] = longest_jams['title'] + ' - ' + longest_jams['city'] + ", " + longest_jams['year.year']


In [11]:
table = dbc.Table.from_dataframe(longest_jams, hover= True, color="secondary", size = "sm", bordered=True)

Table(children=[Thead([Tr([Th(children='artist', colSpan=1), Th(children='venue.location', colSpan=1), Th(children='title', colSpan=1), Th(children='duration_hms', colSpan=1), Th(children='display_date', colSpan=1), Th(children='track_position', colSpan=1)])]), Tbody([Tr([Td('Phish'), Td('Alpharetta, GA'), Td('TWEEZER'), Td('33:50'), Td('2021-08-01'), Td(13.0)]), Tr([Td('Goose'), Td('Raleigh, NC'), Td('MADHUVAN'), Td('29:04'), Td('2022-06-15'), Td(8.0)]), Tr([Td('Phish'), Td('Alpharetta, GA'), Td('CHALK DUST TORTURE'), Td('26:08'), Td('2021-07-31'), Td(9.0)]), Tr([Td('Phish'), Td('Charlestone, SC'), Td('SIMPLE'), Td('23:45'), Td('2022-06-01'), Td(12.0)]), Tr([Td('Phish'), Td('Charlestone, SC'), Td('SET YOUR SOUL FREE'), Td('22:30'), Td('2022-05-31'), Td(1.0)]), Tr([Td('Goose'), Td('Raleigh, NC'), Td('DRIVE'), Td('21:09'), Td('2022-06-15'), Td(12.0)]), Tr([Td('Phish'), Td('Raleigh, NC'), Td('RISE/COME TOGETHER'), Td('20:51'), Td('2022-07-29'), Td(9.0)]), Tr([Td('Goose'), Td('Raleigh, NC

In [ ]:
jams = px.bar(data_frame=longest_jams.sort_values("duration", ascending=True),
              x = 'minutes',
              y = 'title_year_city',
              color = 'artist',
              title = "Longest Jams You've Seen",
              hover_data = ['duration_hms', 'display_date', 'venue.name', "venue.location",],
              labels = {'year.year':'Year',
                        'title_year_city':'Song',
                        'artist':'Artist',
                        'minutes':'Minutes',
                        'duration_hms':'Duration',
                        'venue.name':'Venue',
                        'venue.location': 'City/State',
                        'display_date':'Date'}
            )
#jams.update_layout(xaxis={'categoryorder':'total ascending'})
jams

In [ ]:
len(my_shows)

### Garbage Can

In [17]:
artists = unique_shows['artist'].unique()
artists

array(['Widespread Panic', 'Phish', 'Goose'], dtype=object)

In [15]:
unique_shows.head()

,index,artist_id,venue_id,tour_id,uuid,id,date,avg_rating,avg_duration,display_date,venue.shows_at_venue,venue.latitude,venue.longitude,venue.name,venue.location,venue.slug,venue.past_names,venue.id,year.year,artist,source_uuid,show_uuid,track_position,duration,title,slug,mp3_url
0,45975,5,84504.0,2272.0,1ef462cc-7647-878f-78b0-4a7f635b5c52,24118668,2019-04-27T00:00:00Z,0.0,8877.0,2019-04-27,4.0,32.888561,-80.007505,Riverfront Park,"North Charleston, South Carolina",riverfront-park,NaN,84504.0,2019,Widespread Panic,7001c916-e4f3-caac-17c5-b471fe203953,1ef462cc-7647-878f-78b0-4a7f635b5c52,2.0,320.0,TAIL DRAGGER,tail-dragger,https://www.panicstream.com/streams/wsp/2019_0...
1,46858,5,96983.0,2292.0,8caf7ca3-21e9-d1e6-4ad3-760a2565e2b3,75477000,2021-07-16T00:00:00Z,0.0,10696.0,2021-07-16,3.0,34.225726,-77.944710,Live Oak Bank Pavilion,"Wilmington, North Carolina",live-oak-bank-pavilion,NaN,96983.0,2021,Widespread Panic,25e399c3-0ac0-3090-8e98-5a97b92f4ead,8caf7ca3-21e9-d1e6-4ad3-760a2565e2b3,2.0,182.0,A OF D,a-of-d,https://www.panicstream.com/streams/wsp/2021_0...
2,46880,5,96983.0,2292.0,f7f33882-abc8-314f-5cad-33811c74525e,75564270,2021-07-17T00:00:00Z,0.0,9553.0,2021-07-17,3.0,34.225726,-77.944710,Live Oak Bank Pavilion,"Wilmington, North Carolina",live-oak-bank-pavilion,NaN,96983.0,2021,Widespread Panic,af04f99a-3c90-c4c7-891d-0d05be6752e2,f7f33882-abc8-314f-5cad-33811c74525e,2.0,644.0,RIBS AND WHISKEY,ribs-and-whiskey,https://www.panicstream.com/streams/wsp/2021_0...
3,46901,5,96983.0,2292.0,ff62abb3-8327-93df-5db5-d02a682b375b,75645487,2021-07-18T00:00:00Z,0.0,9951.0,2021-07-18,3.0,34.225726,-77.944710,Live Oak Bank Pavilion,"Wilmington, North Carolina",live-oak-bank-pavilion,NaN,96983.0,2021,Widespread Panic,a95fb7c3-183c-1775-c225-3ff7ff6efe18,ff62abb3-8327-93df-5db5-d02a682b375b,2.0,482.0,TALL BOY,tall-boy,https://www.panicstream.com/streams/wsp/2021_0...
4,46966,5,96439.0,2292.0,715e7c96-0def-9781-9bae-0b1a53510b62,77427988,2021-08-08T00:00:00Z,0.0,10164.0,2021-08-08,3.0,35.600945,-82.554015,ExploreAsheville.com Arena,"Asheville, North Carolina",exploreashevillecom-arena,NaN,96439.0,2021,Widespread Panic,e8795c4b-f8c2-19d2-a94a-cfc0090f5fbb,715e7c96-0def-9781-9bae-0b1a53510b62,2.0,657.0,COCONUT,coconut,https://www.panicstream.com/streams/wsp/2021_0...


In [12]:
unique_shows = my_shows.drop_duplicates(subset = ['uuid']).reset_index()
artist_options = []
for i in range(len(unique_shows)):
     date = unique_shows.loc[i, 'display_date']
     artist = unique_shows.loc[i, 'artist']
     location = unique_shows.loc[i, 'venue.location']
     uuid = unique_shows.loc[i, 'uuid']
     artist_options.append({"label": f"{date} / {artist} / {location}", "value": uuid})

In [ ]:
artist_options[1]

In [ ]:
unique_shows.loc[1, 'artist']

In [ ]:
my_shows.head()

In [39]:
def create_show_list(df: pd.DataFrame, artist_name: str):
    show_list = []
    df_new = df[df['artist'] == artist_name].reset_index(drop = True).copy()
    
    for i in range(len(df_new)):
        venue = df_new.loc[i, "venue.name"]
        date = df_new.loc[i, "display_date"]
        location = df_new.loc[i, "venue.location"]
        uuid = df_new.loc[i, "uuid"]
        
        show_list.append(
                {"label": f"{date} / {location} / {venue}", "value": uuid}
            )
    return show_list
            

In [26]:
unique_shows = (
    my_shows.drop_duplicates(subset=["uuid"])
    .sort_values("date", ascending=False)
    .reset_index()
)
unique_shows

,index,artist_id,venue_id,tour_id,uuid,id,date,avg_rating,avg_duration,display_date,venue.shows_at_venue,venue.latitude,venue.longitude,venue.name,venue.location,venue.slug,venue.past_names,venue.id,year.year,artist,source_uuid,show_uuid,track_position,duration,title,slug,mp3_url
0,83062,259,104892.0,NaN,2acafa23-1821-e5dd-f2b8-a680c73a92db,114673951,2022-10-15T00:00:00Z,0.000,10090.0,2022-10-15,1.0,NaN,NaN,Southside Ballroom,"Dallas, TX",southside-ballroom,NaN,104892.0,2022,Goose,831e10c7-344c-b0b9-4e42-3d307cd63a89,2acafa23-1821-e5dd-f2b8-a680c73a92db,1.0,757.0,EVERYTHING MUST GO,01-everything-must-goflac,https://archive.org/download/goose2022-10-15.I...
1,81217,4,83367.0,2546.0,821f67a4-2a5c-85e5-6fdd-4503b6cf7bbc,108862719,2022-07-29T00:00:00Z,7.940,9671.0,2022-07-29,11.0,35.772096,-78.638615,Walnut Creek,"Raleigh, NC",walnut-creek,NaN,83367.0,2022,Phish,29b2a0dc-6be1-1a8d-8966-bb6473bcca5f,821f67a4-2a5c-85e5-6fdd-4503b6cf7bbc,1.0,510.0,THE MOMA DANCE,the-moma-dance,https://phish.in/audio/000/035/785/35785.mp3
2,82802,259,102862.0,NaN,b8a97c8d-8850-6f9b-6b6a-792b067c8c10,105540245,2022-06-15T00:00:00Z,0.000,9880.0,2022-06-15,1.0,NaN,NaN,Red Hat Amphitheater,"Raleigh, NC",red-hat-amphitheater,NaN,102862.0,2022,Goose,f10e11c3-4dcf-3e2c-1dc4-f09138f6f526,b8a97c8d-8850-6f9b-6b6a-792b067c8c10,2.0,789.0,HUNGERSITE,hungersite,https://archive.org/download/goose2022-06-15.d...
3,80948,4,102176.0,2544.0,df8da48f-7c7c-0273-ea9d-9b7bc877c6d1,104678352,2022-06-01T00:00:00Z,7.412,10447.0,2022-06-01,2.0,32.861670,-79.902020,Credit One Stadium,"Charlestone, SC",credit-one-stadium,NaN,102176.0,2022,Phish,6f425f02-ef1b-0fee-61eb-c663c2d27796,df8da48f-7c7c-0273-ea9d-9b7bc877c6d1,1.0,389.0,COOL IT DOWN,cool-it-down,https://phish.in/audio/000/035/509/35509.mp3
4,80929,4,102176.0,2544.0,b8b481fd-bc0d-f9ac-7202-5e4185a163ec,104610401,2022-05-31T00:00:00Z,6.992,10224.0,2022-05-31,2.0,32.861670,-79.902020,Credit One Stadium,"Charlestone, SC",credit-one-stadium,NaN,102176.0,2022,Phish,7e37ad42-f51e-48c9-e66f-7a35ef6ad772,b8b481fd-bc0d-f9ac-7202-5e4185a163ec,1.0,1350.0,SET YOUR SOUL FREE,set-your-soul-free,https://phish.in/audio/000/035/490/35490.mp3
5,47602,5,NaN,NaN,0be926ad-7264-faa4-b9d4-807d2532f495,102680158,2022-05-07T00:00:00Z,0.000,9933.0,2022-05-07,NaN,NaN,NaN,None,None,None,NaN,NaN,2022,Widespread Panic,c05a7f7c-4c60-da6b-2877-34b47773579f,0be926ad-7264-faa4-b9d4-807d2532f495,1.0,405.0,SURPRISE VALLEY,surprise-valley,https://www.panicstream.com/streams/wsp/2022_0...
6,47580,5,NaN,NaN,fb84061b-5564-3f6a-b6f5-ee0ffeecbb55,102680157,2022-05-06T00:00:00Z,0.000,10492.0,2022-05-06,NaN,NaN,NaN,None,None,None,NaN,NaN,2022,Widespread Panic,345951d7-0c4b-7984-0ef9-b4f30d5e0373,fb84061b-5564-3f6a-b6f5-ee0ffeecbb55,1.0,433.0,JACK,jack,https://www.panicstream.com/streams/wsp/2022_0...
7,47534,5,84513.0,2292.0,5cee6580-4d79-8cdd-1f0a-30b7fcca66aa,98801864,2022-03-26T00:00:00Z,0.000,10215.0,2022-03-26,17.0,29.894691,-81.314517,St. Augustine Amphitheatre,"St. Augustine, Florida",st-augustine-amphitheatre,NaN,84513.0,2022,Widespread Panic,3d9fe9b4-e628-00b8-689c-f7867a638543,5cee6580-4d79-8cdd-1f0a-30b7fcca66aa,1.0,424.0,RIBS AND WHISKEY,ribs-and-whiskey,https://www.panicstream.com/streams/wsp/2022_0...
8,47429,5,84568.0,2292.0,cd37e7c4-1c5a-b17f-30ea-515189dd876e,96312490,2022-02-20T00:00:00Z,0.000,9574.0,2022-02-20,17.0,34.000710,-81.034814,Township Auditorium,"Columbia, South Carolina",township-auditorium,NaN,84568.0,2022,Widespread Panic,bce9d79d-be16-d605-2771-ba5d52c8d24f,cd37e7c4-1c5a-b17f-30ea-515189dd876e,2.0,207.0,NO SUGAR TONIGHT,no-sugar-tonight,https://www.panicstream.com/streams/wsp/2022_0...
9,47408,5,84568.0,2292.0,8a890c21-2732-cbbe-a596-8d8958485e27,96312489,2022-02-19T00:00:00Z,0.000,11364.0,2022-02-19,17.0,34.000710,-81.034814,Township Auditorium,"Columbia, South Carolina",township-auditorium,NaN,84568.0,2022,Widespread Panic,40558fea-187e-57ec-8da9-1b2994c4026d,8a890c21-2732-cbbe-a596-8d8958485e27,2.0,233.0,SOMETIMES,sometimes,https:/

In [35]:
unique_shows[unique_shows['artist'] == "Phish"].reset_index(drop = True).copy()

,index,artist_id,venue_id,tour_id,uuid,id,date,avg_rating,avg_duration,display_date,venue.shows_at_venue,venue.latitude,venue.longitude,venue.name,venue.location,venue.slug,venue.past_names,venue.id,year.year,artist,source_uuid,show_uuid,track_position,duration,title,slug,mp3_url
0,81217,4,83367.0,2546.0,821f67a4-2a5c-85e5-6fdd-4503b6cf7bbc,108862719,2022-07-29T00:00:00Z,7.940,9671.0,2022-07-29,11.0,35.772096,-78.638615,Walnut Creek,"Raleigh, NC",walnut-creek,NaN,83367.0,2022,Phish,29b2a0dc-6be1-1a8d-8966-bb6473bcca5f,821f67a4-2a5c-85e5-6fdd-4503b6cf7bbc,1.0,510.0,THE MOMA DANCE,the-moma-dance,https://phish.in/audio/000/035/785/35785.mp3
1,80948,4,102176.0,2544.0,df8da48f-7c7c-0273-ea9d-9b7bc877c6d1,104678352,2022-06-01T00:00:00Z,7.412,10447.0,2022-06-01,2.0,32.861670,-79.902020,Credit One Stadium,"Charlestone, SC",credit-one-stadium,NaN,102176.0,2022,Phish,6f425f02-ef1b-0fee-61eb-c663c2d27796,df8da48f-7c7c-0273-ea9d-9b7bc877c6d1,1.0,389.0,COOL IT DOWN,cool-it-down,https://phish.in/audio/000/035/509/35509.mp3
2,80929,4,102176.0,2544.0,b8b481fd-bc0d-f9ac-7202-5e4185a163ec,104610401,2022-05-31T00:00:00Z,6.992,10224.0,2022-05-31,2.0,32.861670,-79.902020,Credit One Stadium,"Charlestone, SC",credit-one-stadium,NaN,102176.0,2022,Phish,7e37ad42-f51e-48c9-e66f-7a35ef6ad772,b8b481fd-bc0d-f9ac-7202-5e4185a163ec,1.0,1350.0,SET YOUR SOUL FREE,set-your-soul-free,https://phish.in/audio/000/035/490/35490.mp3
3,80221,4,83399.0,2490.0,819e3062-1b81-250e-1c19-6b8f66e87718,77352940,2021-08-08T00:00:00Z,7.968,10912.0,2021-08-08,29.0,40.005028,-85.933815,Deer Creek,"Noblesville, IN",deer-creek,NaN,83399.0,2021,Phish,0071c02d-800c-68dd-7dd9-c87315336887,819e3062-1b81-250e-1c19-6b8f66e87718,1.0,437.0,SIGMA OASIS,sigma-oasis,https://phish.in/audio/000/034/728/34728.mp3
4,80127,4,83365.0,2490.0,13367854-2d61-402d-4e35-ac76af4042cd,76783808,2021-08-01T00:00:00Z,8.854,10420.0,2021-08-01,14.0,34.055255,-84.306514,Verizon Wireless Amphitheatre at Encore Park,"Alpharetta, GA",verizon-wireless-amphitheatre-at-encore-park,NaN,83365.0,2021,Phish,73faf4d1-9176-a3e5-62ec-35bc897d4baf,13367854-2d61-402d-4e35-ac76af4042cd,1.0,182.0,BURIED ALIVE,buried-alive,https://phish.in/audio/000/034/650/34650.mp3
5,80108,4,83365.0,2490.0,4a5d66c7-1c9b-6eda-e198-1bd3f4cdcfd6,76701358,2021-07-31T00:00:00Z,8.264,10418.0,2021-07-31,14.0,34.055255,-84.306514,Verizon Wireless Amphitheatre at Encore Park,"Alpharetta, GA",verizon-wireless-amphitheatre-at-encore-park,NaN,83365.0,2021,Phish,9f4121a4-4cc6-c566-855b-1e532d640f6b,4a5d66c7-1c9b-6eda-e198-1bd3f4cdcfd6,1.0,557.0,SAND,sand,https://phish.in/audio/000/034/631/34631.mp3
6,79853,4,83022.0,2408.0,72507b40-4b02-fc82-109a-b919566976f4,34605838,2019-12-07T00:00:00Z,8.502,9943.0,2019-12-07,9.0,32.865495,-80.022426,North Charleston Coliseum,"North Charleston, SC",north-charleston-coliseum,NaN,83022.0,2019,Phish,fb90559c-e2b7-8978-23c1-e4cd7757e15d,72507b40-4b02-fc82-109a-b919566976f4,1.0,971.0,FLUFFHEAD,fluffhead,https://phish.in/audio/000/034/232/34232.mp3
7,79827,4,83022.0,2408.0,e588f4ea-be66-3924-419e-3085fff17a4e,34533909,2019-12-06T00:00:00Z,8.456,11357.0,2019-12-06,9.0,32.865495,-80.022426,North Charleston Coliseum,"North Charleston, SC",north-charleston-coliseum,NaN,83022.0,2019,Phish,07e93e0f-9753-4f30-deed-ff6c24f429bc,e588f4ea-be66-3924-419e-3085fff17a4e,1.0,112.0,CAROLINA,carolina,https://phish.in/audio/000/034/206/34206.mp3
8,79281,4,83332.0,2201.0,11005e2a-46b7-288a-d0fb-5281e6730f51,24119263,2019-06-21T00:00:00Z,8.768,9736.0,2019-06-21,7.0,35.327573,-80.711115,Blockbuster Pavilion,"Charlotte, NC",blockbuster-pavilion,NaN,83332.0,2019,Phish,52617854-6052-70a4-9ca3-d4b57cce03a4,11005e2a-46b7-288a-d0fb-5281e6730f51,1.0,321.0,HAVE MERCY,have-mercy,https://phish.in/audio/000/033/642/33642.mp3


In [40]:
test = create_show_list(df = unique_shows, artist_name="Phish")

In [41]:
test

[{'label': '2022-07-29 / Raleigh, NC / Walnut Creek',
  'value': '821f67a4-2a5c-85e5-6fdd-4503b6cf7bbc'},
 {'label': '2022-06-01 / Charlestone, SC / Credit One Stadium',
  'value': 'df8da48f-7c7c-0273-ea9d-9b7bc877c6d1'},
 {'label': '2022-05-31 / Charlestone, SC / Credit One Stadium',
  'value': 'b8b481fd-bc0d-f9ac-7202-5e4185a163ec'},
 {'label': '2021-08-08 / Noblesville, IN / Deer Creek',
  'value': '819e3062-1b81-250e-1c19-6b8f66e87718'},
 {'label': '2021-08-01 / Alpharetta, GA / Verizon Wireless Amphitheatre at Encore Park',
  'value': '13367854-2d61-402d-4e35-ac76af4042cd'},
 {'label': '2021-07-31 / Alpharetta, GA / Verizon Wireless Amphitheatre at Encore Park',
  'value': '4a5d66c7-1c9b-6eda-e198-1bd3f4cdcfd6'},
 {'label': '2019-12-07 / North Charleston, SC / North Charleston Coliseum',
  'value': '72507b40-4b02-fc82-109a-b919566976f4'},
 {'label': '2019-12-06 / North Charleston, SC / North Charleston Coliseum',
  'value': 'e588f4ea-be66-3924-419e-3085fff17a4e'},
 {'label': '2019

In [20]:
unique_shows = (
    my_shows.drop_duplicates(subset=["uuid"])
    .sort_values("date", ascending=False)
    .reset_index()
)
artists = unique_shows["artist"].unique()
phish_options = []
panic_options = []
goose_options = []
for artist in artists:
    tmp = unique_shows[unique_shows["artist"] == artist].copy()
    for i in range(len(tmp)):
        venue = unique_shows.loc[i, "venue.name"]
        date = unique_shows.loc[i, "display_date"]
        location = unique_shows.loc[i, "venue.location"]
        uuid = unique_shows.loc[i, "uuid"]
        if artist == "Phish":
            phish_options.append(
                {"label": f"{date} / {location} / {venue}", "value": uuid}
            )
        elif artist == "Widespread Panic":
            panic_options.append(
                {"label": f"{date} / {location} / {venue}", "value": uuid}
            )
        else:
            goose_options.append(
                {"label": f"{date} / {location} / {venue}", "value": uuid}
            )

In [21]:
phish_options

[{'label': '2022-10-15 / Dallas, TX / Southside Ballroom',
  'value': '2acafa23-1821-e5dd-f2b8-a680c73a92db'},
 {'label': '2022-07-29 / Raleigh, NC / Walnut Creek',
  'value': '821f67a4-2a5c-85e5-6fdd-4503b6cf7bbc'},
 {'label': '2022-06-15 / Raleigh, NC / Red Hat Amphitheater',
  'value': 'b8a97c8d-8850-6f9b-6b6a-792b067c8c10'},
 {'label': '2022-06-01 / Charlestone, SC / Credit One Stadium',
  'value': 'df8da48f-7c7c-0273-ea9d-9b7bc877c6d1'},
 {'label': '2022-05-31 / Charlestone, SC / Credit One Stadium',
  'value': 'b8b481fd-bc0d-f9ac-7202-5e4185a163ec'},
 {'label': '2022-05-07 / None / None',
  'value': '0be926ad-7264-faa4-b9d4-807d2532f495'},
 {'label': '2022-05-06 / None / None',
  'value': 'fb84061b-5564-3f6a-b6f5-ee0ffeecbb55'},
 {'label': '2022-03-26 / St. Augustine, Florida / St. Augustine Amphitheatre',
  'value': '5cee6580-4d79-8cdd-1f0a-30b7fcca66aa'},
 {'label': '2022-02-20 / Columbia, South Carolina / Township Auditorium',
  'value': 'cd37e7c4-1c5a-b17f-30ea-515189dd876e'}